In [4]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import statsmodels.api as sm

from sklearn.linear_model import HuberRegressor
# pd.set_option('display.max_rows', None)
# more

In [5]:
df = pd.read_csv('data/factors_1985.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])
df.drop(columns=['sic2'], inplace=True)

In [6]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,risk_premium,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf
0,10137,1985-01-31,1419646.25,0.107145,0.011480,0.243134,13.162142,0.024286,0.146054,0.001702,...,-6.7457,3.281008,2.508235,0.534629,-0.029905,0.0776,-0.0412,0.0118,0.001636,7.99
1,10145,1985-01-31,2864500.50,1.092789,1.194188,0.246623,14.078514,0.033520,-0.181981,-0.007194,...,4.9935,3.281008,2.508235,0.534629,-0.029905,0.0776,-0.0412,0.0118,0.001636,7.99
2,10161,1985-01-31,1114782.50,1.641435,2.694309,-0.039232,12.485543,0.037102,-0.183451,-0.044118,...,8.1038,3.281008,2.508235,0.534629,-0.029905,0.0776,-0.0412,0.0118,0.001636,7.99
3,10225,1985-01-31,3539211.25,0.533991,0.285147,0.230335,13.111023,0.024247,0.042925,0.015810,...,-8.8282,3.281008,2.508235,0.534629,-0.029905,0.0776,-0.0412,0.0118,0.001636,7.99
4,10233,1985-01-31,1834917.50,0.941089,0.885648,-0.126162,14.589921,0.037974,-0.042444,0.069915,...,-6.6639,3.281008,2.508235,0.534629,-0.029905,0.0776,-0.0412,0.0118,0.001636,7.99


In [7]:
df= df[~np.isnan(df['bm'])]
df =df[~np.isnan(df['mvel1'])]

In [8]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [9]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\2840026024.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\2840026024.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1'))

In [10]:
#Standardize all independent variables

stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()

df[features]=df.groupby('DATE')[features].rank(pct=True)

df[features] = stdSc.fit_transform(df[features].astype(float))
#Multiply by 2 and substract 1 to get ranks in interval [-1,1] 
# df[features] = 2*df[features] - 1

In [76]:
df['risk_premium'] = df['risk_premium'] / 100
df['macro_mkt-rf'] = df['macro_mkt-rf'] / 100
df['macro_tbl'] = df['macro_tbl'] / 100



In [74]:

def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [78]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                #    test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

predictions = []
y_val_list = []
dates = []
r2_train_list = []
r2_val_list = []

for train_index, val_index in tscv.split(X, first_split_date= datetime.date(2005,1,31), second_split_date= datetime.date(2007,1,31)):



    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=10000)
   
    huber.fit(X_train, y_train)
    y_pred_train = huber.predict(X_train)
    r2_train = calculate_r2(y_train['risk_premium'], y_pred_train, in_sample=True)
    r2_train_list.append(r2_train)
   
    
    y_pred_val = huber.predict(X_val)
    r2_val = calculate_r2(y_val['risk_premium'], y_pred_val, in_sample=False, benchmark=np.mean(y_val))
    r2_val_list.append(r2_val)
    predictions.append(y_pred_val)
    dates.append(y_val.index)
    y_val_list.append(y_val)

    print(f'R2 {y_train.index[0][0].date()} - {y_train.index[-1][0].date()} training set {r2_train}')
    print(f'R2 {y_val.index[0][0].date()} - {y_val.index[-1][0].date()} validation set {r2_val}')
    print('---')

predictions_full = np.concatenate(predictions, axis=0)
y_test_list_all = np.concatenate(y_val_list, axis=0)
dates_all = np.concatenate(dates, axis=0)
print('')
r2_train_full = np.mean(r2_train)
print(f'R2 training set {r2_train_full}')
r2_val_full = np.mean(r2_val)
print(f'R2 validation set {r2_val_full}')

Train period: 2000-01-31 - 2005-01-31 ,val period: 2005-01-31 - 2007-01-31 # train records 85693 ,# val records 45111
Train period: 2001-01-31 - 2006-01-31 ,val period: 2006-01-31 - 2008-01-31 # train records 88894 ,# val records 47991
Train period: 2002-01-31 - 2007-01-31 ,val period: 2007-01-31 - 2009-01-31 # train records 96182 ,# val records 46419
Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 # train records 105203 ,# val records 38716
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 # train records 109510 ,# val records 34937
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 # train records 108297 ,# val records 39748
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 # train records 104877 ,# val records 43094
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 # train records 102934 ,# val records 45393
Train period: 2008-01-31 - 2013-01-31 ,val period: 

c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 2000-01-31 - 2004-12-31 training set 0.014802008930585697
R2 2005-01-31 - 2006-12-29 validation set -0.011052380427150732
---


c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 2001-01-31 - 2005-12-30 training set 0.009546046902940808
R2 2006-01-31 - 2007-12-31 validation set -0.04893533854805021
---


...

R2 2003-01-31 - 2007-12-31 training set 0.0597669105495523
R2 2008-01-31 - 2010-01-29 validation set 0.03553566236572869
---


c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 2004-02-27 - 2009-01-30 training set 0.007481444329897502
R2 2009-02-27 - 2010-12-31 validation set -0.10036942046215747
---


c:\Users\drebi\miniconda3\envs\statclass\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:

tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_large[features]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
y_test_list =[]
dates = []
dic_r2_all = {}

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    reg_huber = HuberRegressor()

    reg_huber.fit(X_train, y_train)
    y_pred_train = reg_huber.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = reg_huber.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = reg_huber.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))



r2_val_large = np.mean(y_val_list)
print(r2_val_large)

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_small[features]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

predictions_top = []
y_train_list_top =[]
dates_top = []
dic_r2_all_top = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    reg_huber = HuberRegressor()

    reg_huber.fit(X_train, y_train)
    y_pred_train = reg_huber.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = reg_huber.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = reg_huber.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))



r2_val_small = np.mean(y_val_list)
print(r2_val_small)

In [ ]:
r2_val_small

In [ ]:
chart = np.array([[r2_val_full],
                  [r2_val_large ],
                  [r2_val_small]])

r2_lm = pd.DataFrame(chart, columns=['Huber Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

In [ ]:
r2_lm.to_csv(r'r2_linear_model.csv')

In [ ]:
predictions_all_top= np.concatenate(preds_train, axis=0)
y_train_list_all_top= np.concatenate(y_train_list, axis=0) 
dates_all_top= np.concatenate(dates_train, axis=0)

R2OOS_LR_top = 1-sum(pow(y_test_list_all_top-predictions_all_top,2))/sum(pow(y_test_list_all_top,2))

In [44]:

yhat = predictions_full.tolist()
y_true = y_test_list_all.tolist()
i = dates_all.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })


results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[35:40]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results.head()


yhat                 y_true       date
MonthYear id                                               
2005-01   10065 -1.791831   [-4.766600131988525] 2005-01-31
          10078 -3.419899  [-21.589500427246094] 2005-01-31
          10085 -0.547494  [-11.270600318908691] 2005-01-31
          10104 -4.065423   [-2.115600109100342] 2005-01-31
          10107 -2.086116   [-4.126699924468994] 2005-01-31

In [45]:
# results['yhat'] = results['yhat'].apply(lambda x: x[0])
results['y_true'] = results['y_true'].apply(lambda x: x[0])

In [46]:
data = df[['mvel12', 'macro_tbl', 'macro_svar']].copy()
data.reset_index(inplace=True)
data['permno2'] = data['permno2'].astype('str')
data['MonthYear'] = data['DATE2'].dt.to_period('M')
data.drop('DATE2', axis=1, inplace=True)
data.rename(columns={'permno2': 'id'}, inplace=True)
data.rename(columns={'mvel12': 'market_cap'}, inplace=True)
data.rename(columns={'macro_tbl': 'risk_free_rate'}, inplace=True)
data = data.set_index(['MonthYear','id'])


In [47]:
bigdata = pd.merge(results, data,left_index=True, right_index=True)
bigdata.reset_index(inplace=True)
bigdata

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar
0,2005-01,10065,-1.791831,-4.7666,2005-01-31,1.098564e+06,-0.338726,-0.338726
1,2005-01,10078,-3.419899,-21.5895,2005-01-31,1.820667e+07,-0.338726,-0.338726
2,2005-01,10085,-0.547494,-11.2706,2005-01-31,1.550179e+06,-0.338726,-0.338726
3,2005-01,10104,-4.065423,-2.1156,2005-01-31,7.168516e+07,-0.338726,-0.338726
4,2005-01,10107,-2.086116,-4.1267,2005-01-31,1.346528e+08,-0.338726,-0.338726
...,...,...,...,...,...,...,...,...
716613,2021-01,93393,-4.568203,-5.4710,2021-01-29,1.050616e+06,-0.743094,-0.743094
716614,2021-01,93419,-4.124894,-2.4747,2021-01-29,3.654523e+06,-0.743094,-0.743094
716615,2021-01,93423,-5.418342,0.2333,2021-01-29,2.897716e+06,-0.743094,-0.743094
716616,2021-01,93427,-3.508312,1.6799,2021-01-29,2.866019e+06,-0.743094,-0.743094


In [48]:
bigdata['returns'] = bigdata['y_true'] + bigdata['risk_free_rate']
bigdata

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns
0,2005-01,10065,-1.791831,-4.7666,2005-01-31,1.098564e+06,-0.338726,-0.338726,-5.105326
1,2005-01,10078,-3.419899,-21.5895,2005-01-31,1.820667e+07,-0.338726,-0.338726,-21.928227
2,2005-01,10085,-0.547494,-11.2706,2005-01-31,1.550179e+06,-0.338726,-0.338726,-11.609327
3,2005-01,10104,-4.065423,-2.1156,2005-01-31,7.168516e+07,-0.338726,-0.338726,-2.454326
4,2005-01,10107,-2.086116,-4.1267,2005-01-31,1.346528e+08,-0.338726,-0.338726,-4.465426
...,...,...,...,...,...,...,...,...,...
716613,2021-01,93393,-4.568203,-5.4710,2021-01-29,1.050616e+06,-0.743094,-0.743094,-6.214094
716614,2021-01,93419,-4.124894,-2.4747,2021-01-29,3.654523e+06,-0.743094,-0.743094,-3.217794
716615,2021-01,93423,-5.418342,0.2333,2021-01-29,2.897716e+06,-0.743094,-0.743094,-0.509794
716616,2021-01,93427,-3.508312,1.6799,2021-01-29,2.866019e+06,-0.743094,-0.743094,0.936806


In [49]:
bigdata['MonthYear1'] = bigdata['MonthYear'].copy()
bigdata['MonthYear'] = bigdata['MonthYear'].astype('int64')
bigdata['NumMonth'] = bigdata['MonthYear'] - 251
bigdata['NumMonth'].unique()

array([169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
       234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259,
       260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272,
       273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285,
       286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298,
       299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
       312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324,
       325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337,
       338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 34

In [50]:

for i in bigdata['NumMonth'].unique():
    globals()['df_' + str(i)] = bigdata[bigdata['NumMonth'] == i]

In [51]:

for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
    
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["DecileRank"]=pd.qcut(globals()['df_' + str(i)]['rank'].values, 10, labels = False)

#Drop normal rank, retain only decile ranks 
for i in bigdata["NumMonth"].unique():
     globals()['df_' + str(i)].drop('rank', axis=1, inplace=True)

C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\1518367074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\1518367074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\1518367074.py:2: SettingWithCopyWarning: 
A value is try

In [52]:

for i in bigdata["NumMonth"].unique():
    for j,g in globals()['df_' + str(i)].groupby('DecileRank'):
        globals()['df_' + str(i)+ "_" + str(j)] =  g

In [53]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)] = pd.concat([globals()['df_1_'+ str(j)], globals()['df_2_'+ str(j)]], axis=0)
    
# Generate 10 Dataframes for the 10 Decile portfolios 0-9: rank_9: top portfolio, rank_0: bottom portfolio
for i in np.arange(2,348,1):
    for j in np.arange(0,10,1):
        globals()['rank_' + str(j)] = pd.concat([globals()['rank_' + str(j)], globals()['df_' + str(i+1)+ "_" + str(j)]], axis = 0)

In [54]:

rank_9

,MonthYear,id,yhat,y_true,date,market_cap,risk_free_rate,macro_svar,returns,MonthYear1,NumMonth,DecileRank
0,252,10078,-2.624856,26.5585,1991-01-31,2.107233e+06,3.130398,3.130398,29.688898,1991-01,1,9
1,252,10104,-3.756695,8.0957,1991-01-31,1.052793e+06,3.130398,3.130398,11.226098,1991-01,1,9
9,252,10401,-3.524148,2.1088,1991-01-31,3.280947e+07,3.130398,3.130398,5.239198,1991-01,1,9
12,252,10604,-3.850705,-9.6278,1991-01-31,1.993750e+07,3.130398,3.130398,-6.497402,1991-01,1,9
14,252,10967,-3.270050,7.0375,1991-01-31,1.325386e+06,3.130398,3.130398,10.167898,1991-01,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...
687402,599,93246,-0.162755,0.6018,2019-12-31,6.160387e+06,-0.645844,-0.645844,-0.044044,2019-12,348,9
687410,599,93268,8.454015,-12.7764,2019-12-31,1.221662e+06,-0.645844,-0.645844,-13.422244,2019-12,348,9
687416,599,93296,0.220877,3.1023,2019-12-31,8.179392e+06,-0.645844,-0.645844,2.456456,2019-12,348,9
687426,599,93330,0.148051,-3.9707,2019-12-31,5.535622e+06,-0.645844,-0.645844,-4.616544,2019-12,348,9


In [55]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]["eq_weights"] = 1/globals()['rank_' + str(j)].groupby('MonthYear')["id"].transform('size')

In [56]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_stock_ew'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["eq_weights"]

In [57]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_stock_ew'] = globals()['rank_' + str(j)]["returns"]*globals()['rank_' + str(j)]["eq_weights"]

In [58]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_ew"].transform('sum')

In [59]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_ew"].transform('sum')

In [60]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_stock_ew'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["eq_weights"]

In [61]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_ew"].transform('sum')

In [62]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)] = globals()['rank_' + str(j)][["MonthYear1", "DecileRank",
                                                                      "excess_return_portfolio_ew",
                                                                      "pred_excess_return_portfolio_ew",
                                                                      "return_portfolio_ew"]]

In [63]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].drop_duplicates()
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].set_index("MonthYear1")

In [64]:
for j in np.arange(0,10,1):
    #Time-series average of realized excess returns
    globals()["ew_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()
    #Time-series average of predicted excess returns
    globals()["ew_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_ew"].mean()
    #Standard deviation of realized excess returns
    globals()["std_ew_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].std()
    #Annualized sharpe ratio of realized excess returns
    globals()["sharpe_ew_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_ew"].std())* np.sqrt(12)

In [65]:

# For the zero-net-investment long-short portfolio the top (long) and bottom(short) decile portfolios are needed

long_monthly = rank_9[["NumMonth",
                       "MonthYear1",
                       "DecileRank", 
                       "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

short_monthly = rank_0[["NumMonth",
                        "MonthYear1",
                        "DecileRank",
                        "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

# Create a column, indication the stategy 
long_monthly["Strategy"]= "long"
short_monthly["Strategy"]= "short"

# Merge to get the zero net investment portfolio
zeronet_monthly= pd.concat([long_monthly, short_monthly])
zeronet_monthly = zeronet_monthly.sort_values(by = ['NumMonth',"Strategy"])

#Create two new columns containing the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["excess_return_zeronet_ew"] =0


In [66]:
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 3]-zeronet_monthly.iloc[i+1, 3]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
        


zeronet_monthly["pred_excess_return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 4]-zeronet_monthly.iloc[i+1, 4]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]


zeronet_monthly["return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 5]-zeronet_monthly.iloc[i+1, 5]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 5]-zeronet_monthly.iloc[i, 5]

C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\3427268320.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.035099730513762' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\3427268320.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.3555980844841868' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]
C:\Users\drebi\AppData\Local\Temp\ipykernel_65236\3427268320.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.2588577888900776' h

In [67]:
#Only the measures at portfolio level are needed
zeronet_monthly = zeronet_monthly[['NumMonth', 
                                   'MonthYear1', 
                                   'excess_return_zeronet_ew',
                                   'pred_excess_return_zeronet_ew',
                                   'return_zeronet_ew',
                                   ]].drop_duplicates()

zeronet_monthly

,NumMonth,MonthYear1,excess_return_zeronet_ew,pred_excess_return_zeronet_ew,return_zeronet_ew
0,1,1991-01,5.035100,1.355598,1.258858
13,1,1991-01,-0.246077,2.678061,-0.246077
518,2,1991-02,-2.483700,-1.426482,-2.258987
531,2,1991-02,-1.118801,2.882720,-1.118801
1061,3,1991-03,5.899116,-1.605643,6.127433
...,...,...,...,...,...
673933,346,2019-10,1.935899,2.341884,1.935899
678334,347,2019-11,-6.180640,-2.127237,-6.172229
678435,347,2019-11,2.641661,2.321323,2.641661
682874,348,2019-12,1.331611,-2.096919,1.341347


In [68]:
#Calculate zero-net portfolio performance measures                                            
#Time-series average of realized excess returns                                             
ew_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_ew"].mean()
#Time-series average of predicted excess returns
ew_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_ew"].mean()
#Standard deviation of realized excess returns
std_ew_zeronet = zeronet_monthly["excess_return_zeronet_ew"].std()
#Annualized sharpe ratio of realized excess returns
sharpe_ew_zeronet = (zeronet_monthly["excess_return_zeronet_ew"].mean()/zeronet_monthly["return_zeronet_ew"].std())* np.sqrt(12)

In [70]:
zeronet_monthly['excess_return_zeronet_ew']

0         5.035100
13       -0.246077
518      -2.483700
531      -1.118801
1061      5.899116
            ...   
673933    1.935899
678334   -6.180640
678435    2.641661
682874    1.331611
682973   -4.763119
Name: excess_return_zeronet_ew, Length: 696, dtype: float64

In [71]:
chart_np = np.array([[ew_mean_pred_return_rank_0, ew_mean_return_rank_0, std_ew_rank_0, sharpe_ew_rank_0],
                     [ew_mean_pred_return_rank_1, ew_mean_return_rank_1, std_ew_rank_1, sharpe_ew_rank_1],
                     [ew_mean_pred_return_rank_2, ew_mean_return_rank_2, std_ew_rank_2, sharpe_ew_rank_2],
                     [ew_mean_pred_return_rank_3, ew_mean_return_rank_3, std_ew_rank_3, sharpe_ew_rank_3],
                     [ew_mean_pred_return_rank_4, ew_mean_return_rank_4, std_ew_rank_4, sharpe_ew_rank_4],
                     [ew_mean_pred_return_rank_5, ew_mean_return_rank_5, std_ew_rank_5, sharpe_ew_rank_5],
                     [ew_mean_pred_return_rank_6, ew_mean_return_rank_6, std_ew_rank_6, sharpe_ew_rank_6],
                     [ew_mean_pred_return_rank_7, ew_mean_return_rank_7, std_ew_rank_7, sharpe_ew_rank_7],
                     [ew_mean_pred_return_rank_8, ew_mean_return_rank_8, std_ew_rank_8, sharpe_ew_rank_8],
                     [ew_mean_pred_return_rank_9, ew_mean_return_rank_9, std_ew_rank_9, sharpe_ew_rank_9],
                     [ew_mean_pred_return_zeronet, ew_mean_return_zeronet, std_ew_zeronet, sharpe_ew_zeronet]])

ew_df = pd.DataFrame(chart_np, columns=['Pred', 'Real', 'Std', 'Sharpe'],
                              index=['Low (L)', '2', '3', '4', '5','6','7','8',"9",'High (H)', "H-L"])

ew_df['Pred'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Pred']], index = ew_df.index)
ew_df['Real'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Real']], index = ew_df.index)
ew_df['Std'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Std']], index = ew_df.index)
ew_df['Sharpe'] = pd.Series([("%.2f" % round(val, 2)) for val in ew_df['Sharpe']], index = ew_df.index)
ew_df

,Pred,Real,Std,Sharpe
Low (L),-6.03%,-2.12%,5.79%,-1.27
2,-5.32%,-1.85%,4.97%,-1.30
3,-4.96%,-1.72%,4.81%,-1.25
4,-4.38%,-1.68%,4.80%,-1.22
5,-3.93%,-1.76%,4.83%,-1.27
6,-2.94%,-1.78%,4.79%,-1.30
7,-2.62%,-1.75%,4.80%,-1.28
8,-2.29%,-1.72%,4.72%,-1.28
9,-1.95%,-1.69%,4.83%,-1.22
High (H),-1.35%,-1.60%,5.06%,-1.10
